In [1]:
import lammps
import pymc3 as pm
import arviz as az
import numpy as np
# import glob
from copy import deepcopy
import matplotlib.pyplot as plt

from lammps_data import LammpsData
from pymatgen.io.vasp import Poscar

In [2]:
def init(structure_data, structure, cs_springs=None):
    """
    Initialises the system from the structure (read in from lammps input) and non-changing parameters.
    
    Args:
        structure (str): Name of lammps input file.
        cs_springs (optional:dict): A dictionary of the bond_coeff values. Key = atom label and values = K
                                    (energy/distance^2) and r0 (equilibrium bond distance) (list).
                                    Default = None.
        structure_data (obj): LammpsData stuctural object containing information on atom_types, bond_types,
                              atoms, bonds, cell_lengths, and tilt_factors.

    Returns:
        lmp (Lammps): Lammps system with structure and specified commands implemented.
    """
    
    lmp = lammps.Lammps(units='metal', style = 'full', args=['-log', 'none', '-screen', 'none'])
    lmp.command('read_data {}'.format(structure))

    lmp.command('group cores type {}'.format(structure_data.type_core()))
    lmp.command('group shells type {}'.format(structure_data.type_shell()))

    if cs_springs:
        lmp.command('pair_style buck/coul/long/cs 10.0')
        lmp.command('pair_coeff * * 0 1 0')

        lmp.command('bond_style harmonic')
        for i, spring in enumerate(cs_springs):
            lmp.command('bond_coeff {} {} {}'.format(i+1,
                                                     cs_springs[spring][0],
                                                     cs_springs[spring][1]))
    else:
        lmp.command('pair_style buck/coul/long 10.0')
        lmp.command('pair_coeff * * 0 1 0')
    
    lmp.command('kspace_style ewald 1e-6')
        
    #setup for minimization
    lmp.command('min_style cg')

    return lmp

In [3]:
def dex(element):
    """
    Finds atom_index number for given element, where core-shell returns shell index.

    Args:
        element (str): element symbol

    Returns:
        atom_index (int): index of atom relating to element. In the case of core-shell atoms
                          this will return the shell index.
    """
    element_list = [e.label for e in structure_data.atom_types]
    for e in element_list:
        if element in e and 'core' not in e:
            atom_index = element_list.index(e)+1
    return atom_index

def update_potentials(**kwargs):
    """
    Unpdates the potentials set by pymc3 into the dictionary for the fitting process.

    Args:
        **kwargs: The parameters to be updated in the fitting process as set with pm.Model.

    Returns:
        None.
    """
    bpp.update(kwargs)
    
def set_potentials(instance):
    """
    Sets the potential for the sepecified Lammps system (changes for each iteration of the potential fit).

    Args:
        instance (Lammps): Lammps system with structure and specified commands implemented.

    Returns:
        None
    """
    for pair in pairs:
        instance.command('pair_coeff {} {} {} {} {}'.format(dex(pair[0]),
                                                            dex(pair[1]),
                                                            bpp['{}_{}_A'.format(pair[0], pair[1])],
                                                            bpp['{}_{}_rho'.format(pair[0], pair[1])],
                                                            bpp['{}_{}_C'.format(pair[0], pair[1])]))

In [4]:
def simfunc(**kwargs):
    """
    Runs a minimization and zero step run for the instance and returns the forces.

    Args:
        **kwargs: Contain data for type of fitting and to what parameters as set with pm.Model.

    Returns:
        out (np.array): x,y,z forces on each atom.
    """
    if min(kwargs.values()) > 0:
        update_potentials(**kwargs)
        out = np.zeros([sum(core_mask),3])
        
        set_potentials(instances)

        instances.command('fix 1 cores setforce 0.0 0.0 0.0')
        instances.command('minimize 1e-25 1e-25 5000 10000')
        instances.command('unfix 1')
        instances.run(0)
        
        out = instances.system.forces[core_mask]
            
    else:
        out = np.ones([sum(core_mask),3])*999999999 # ThisAlgorithmBecomingSkynetCost

    return out

In [5]:
# set up some auxiliary data
params = {}
params['core_shell'] = { 'Li': False, 'Ni': False, 'O': True }
params['charges'] = {'Li': +1.0,
                     'Ni': +3.0,
                     'O': {'core':  +0.960,
                           'shell': -2.960}}
params['masses'] = {'Li': 6.941,
                    'Ni': 58.6934,
                    'O': {'core': 15.0,
                          'shell': 1.0} }

poscar = Poscar.from_file('poscars/POSCAR1')
my_structure = poscar.structure
structure_data = LammpsData.from_structure(my_structure, params)
lammps_file = 'lammps/coords1.lmp'
with open( lammps_file, 'w' ) as f:
    f.write(structure_data.input_string())

pairs = [('Li','O'), ('Ni','O'), ('O','O')]

#Must be in same order given in charges dictionary
cs_springs = {'O' : [65.0, 0.0]} #Set to None if not using any core-shells

bpp_def = {'Li_O_A'   : 632.1018,
           'Li_O_rho' : 0.2906,
           'Li_O_C'   : 0.0,
           'Ni_O_A'   : 1582.5000,
           'Ni_O_rho' : 0.2882,
           'Ni_O_C'   : 0.000,
           'O_O_A'    : 22764.3000,
           'O_O_rho'  : 0.1490,
           'O_O_C'    : 21.7}

bpp = deepcopy(bpp_def)

Found elements: ['Li', 'Ni', 'O']


In [6]:
instances = init(structure_data, lammps_file, cs_springs)
core_mask = structure_data.core_mask()

In [7]:
expected = np.zeros([sum(core_mask),3])

In [ ]:
with pm.Model() as model:
#  pm.TruncatedNormal -- truncated so never tries a negative    pm.Uniform
    Li_O_A      = pm.Normal("Li_O_A",    mu = bpp_def['Li_O_A'],    sd =  50)
    Li_O_rho    = pm.Normal("Li_O_rho",  mu = bpp_def['Li_O_rho'],  sd =  0.05)
#     Li_O_C      = pm.Normal("Li_O_C",    mu = bpp_def['Li_O_C'],    sd =  0.01)
    Ni_O_A      = pm.Normal("Ni_O_A",    mu = bpp_def['Ni_O_A'],    sd =  100)
    Ni_O_rho    = pm.Normal("Ni_O_rho",  mu = bpp_def['Ni_O_rho'],  sd =  0.05)
#     Ni_O_C      = pm.Normal("Ni_O_C",    mu = bpp_def['Ni_O_C'],    sd =  0.05)
    O_O_A       = pm.Normal("O_O_A",     mu = bpp_def['O_O_A'],     sd =  100)
    O_O_rho     = pm.Normal("O_O_rho",   mu = bpp_def['O_O_rho'],   sd =  0.01)
    O_O_C       = pm.Normal("O_O_C",     mu = bpp_def['O_O_C'],     sd =  5)
    
    simulator = pm.Simulator('simulator', simfunc, observed=expected)
    
    trace = pm.sample(step=pm.SMC(ABC=True, epsilon=0.1), draws=2000)
#     trace = pm.sample(step=pm.SMC(ABC=True, epsilon=3000, dist_func="sum_of_squared_distance"), parallel=False, draws=1000)

In [ ]:
az.style.use('arviz-darkgrid')
az.plot_trace(trace)
plt.savefig('coreshell_LiNiO2_trace2.png',dpi=500, bbox_inches = "tight")

In [ ]:
az.plot_posterior(trace, round_to = 3, point_estimate = 'mode')
plt.savefig('coreshell_LiNiO2_mode2.png',dpi=500, bbox_inches = "tight")